# Tuturial 1: Creating and Solving Factor Graphs
In this tutorial you will lear the basics of factor graphs and consepts such as variables, factors, priors and how create solve and explore them.

## What is a Factor Graph

Wikipedia tels us a factor graph is a bipartite graph representing the factorization of a function.
 
If you haven't taken some advanced statistics classes you are probably wondering, bipartite graph?, factorization?, I just want to navigate my robot.
For robotics, factor graphs are the common language used to describe your robot's estimation problem (the function) in a way both humans and computers can understand. 
The estimation problem can be anything from robot localization, structure from motion, calibration to full PoseSLAM with parametric or non-parametric measurements and beliefs.
A factor graph is a graphical model with two types of nodes, variables and factors, connected by edges between the variables and factors (bipartite graph). 
Variables represent the unknown random variables in the estimation problem, such as vehicle or landmark positions, sensor calibration parameters, and more.
Factors represent the algebraic interaction between particular variables, such as wheel odometry, which is captured through edges. 
Breaking down the complex system describing your robot to a graph of variables and factors is the factorization of the function decribing your system, such as the position and orientation (pose) of your robot at any given time. 
This factorisation allows us to solve the optimization problem for every variable given every measurement described by the factors.
See Caesar.jl docs [Graph Concepts](https://juliarobotics.org/Caesar.jl/latest/concepts/concepts/#Graph-Concepts) for more detail.

### Lets build a factor graph

First we will need some packages: 
- `GraphPlot` - for visualizing graphs.
- `DistributedFactorGraphs` - standardized API for interacting with factor graphs.
- `IncrementalInference` - Optimization routines for incremental non-parametric and parametric solutions based on factor graphs.
- `RoME` - Robot Motion Estimate: Tools, Variables, and Factors for SLAM in robotics.

In [ ]:
using Pkg

Pkg.status()

In [ ]:
using GraphPlot
using DistributedFactorGraphs
using IncrementalInference
using RoME

To create a new factor graph with default settings use the `initfg()` function. 

In [ ]:
fg = initfg();

## Variable and Factors 

Variables, denoted as the larger nodes in the figure below, represent state variables of interest such as vehicle or landmark positions, sensor calibration parameters, and more. Variables are likely hidden values which are not directly observed, but we want to estimate them them from observed data and at least some minimal algebra structure from probabilistic measurement models.

`RoME` provides variables and factors usefull to robotics.
We start with a `Pose2` variable, i.e. position and orientation in two dimensions.
To add variables to our factor graph `fg` we created above, call `addVariable` with the labels (`x0`,`x1`) and type `Pose2`

In [ ]:
addVariable!(fg, :x0, Pose2);
addVariable!(fg, :x1, Pose2);

Lets have a look at the factor graph we have so far using `plotDFG`.

In [ ]:
plotDFG(fg)

Factors, the smaller nodes in the figure, represent the algebraic interaction between particular variables, which is captured through edges. Factors must adhere to the limits of probabilistic models – for example conditional likelihoods capture the likelihood correlations between variables; while priors (unary to one variable) represent absolute information to be introduced. 

Since factors represents a probabilistic interaction between variables we need to specify the distribution our factor will represent. 
Lets look as a odometry factor that connects our two robot poses `x0` and `x1`.

In [ ]:
odo_distribution = MvNormal([1.0, 0.0, pi/2], diagm([0.1, 0.1, 0.01]))
fac_1 = addFactor!(fg, [:x0, :x1], Pose2Pose2(odo_distribution))

plotDFG(fg)

We now have a factor graph with 2 variable and one factor connecting them, but to solve it we need some additional information. 
In this example we need the estimated starting point of our robot.
We use unary factors called priors to represent absolute information to be introduced.


In [ ]:
prior_distribution = MvNormal(zeros(3), diagm([0.5,0.5,0.1]))
addFactor!(fg, [:x0], PriorPose2(prior_distribution))

plotDFG(fg)

### Solving

We now have a graph we can solve.

In [ ]:
solveGraph!(fg);

### Results



In [ ]:
using RoMEPlotting

In [ ]:
#var1 = getVariable(fg, :x1)
pl = plotSLAM2D(fg, drawhist=true, drawPoints=false)

img = SVG("plot2d.svg", 6inch, 4inch)
draw(img, pl)

What is happening


In [ ]:
var = getVariable(fg, :x1)

In [ ]:
getPPESuggested(fg, :x1)

`getPPESuggested` returns a parametric point estimate for the current belief in variable `x1`.

Since the default solver is non-parametric a full belief function is available and can be visualized using `plotKDE`  `#FIXME explain KDE and the solver briefly`



In [ ]:
plotKDE(fg, :x1)

`#FIXME Explain what we see above`

### Adding more poses and a point landmark

`#TODO to close the loop`

In [ ]:

addVariable!(fg, :l1, Point2)

p2br = Pose2Point2BearingRange(Normal(0.0,0.03),Normal(0.5,0.1))
addFactor!(fg, [:x0,:l1], p2br)

addVariable!(fg, :x2, Pose2)
addFactor!(fg, [:x1,:x2], Pose2Pose2(odo_distribution))

addVariable!(fg, :x3, Pose2)
addFactor!(fg, [:x2,:x3], Pose2Pose2(odo_distribution))



#TODO solve again with closed loop
#TODO solve on cloud
#TODO also do parametric
#TODO show the bayes tree and give a basic introduction on the solver